In [4]:
import numpy as np
import heapq
from functools import partial
from collections import namedtuple
import itertools

# Dijkstraアルゴリズムの結果を格納するためのnamedtupleを定義
DijkstraOutput = namedtuple("DijkstraOutput", ["shortest_path", "is_unique", "transitions", "edges"])

class Dijkstra:
    def __init__(self, matrix, neighbourhood_fn="8-grid"):
        self.matrix = matrix
        self.neighbourhood_fn = neighbourhood_fn
        self.x_max, self.y_max = matrix.shape
        # 近傍関数を設定
        self.neighbors_func = partial(self.get_neighbourhood_func(neighbourhood_fn), x_max=self.x_max, y_max=self.y_max)
        # コスト行列を初期化
        self.costs = np.full_like(matrix, 1.0e10)
        self.costs[0][0] = matrix[0][0]
        # 最短経路の数を格納する行列を初期化
        self.num_path = np.zeros_like(matrix)
        self.num_path[0][0] = 1
        # 優先度付きキューを初期化
        self.priority_queue = [(matrix[0][0], (0, 0))]
        # 確定した頂点の集合
        self.certain = set()
        # 遷移情報を格納する辞書
        self.transitions = dict()

    # 近傍関数を取得するためのメソッド
    def get_neighbourhood_func(self, neighbourhood_fn):
        if neighbourhood_fn == "4-grid":
            return self.neighbours_4
        elif neighbourhood_fn == "8-grid":
            return self.neighbours_8
        else:
            raise Exception(f"neighbourhood_fn of {neighbourhood_fn} not possible")

    # 8方向の近傍を返すジェネレータ
    def neighbours_8(self, x, y, x_max, y_max):
        deltas_x = (-1, 0, 1)
        deltas_y = (-1, 0, 1)
        for (dx, dy) in itertools.product(deltas_x, deltas_y):
            x_new, y_new = x + dx, y + dy
            if 0 <= x_new < x_max and 0 <= y_new < y_max and (dx, dy) != (0, 0):
                yield x_new, y_new

    # 4方向の近傍を返すジェネレータ
    def neighbours_4(self, x, y, x_max, y_max):
        for (dx, dy) in [(1, 0), (0, 1), (0, -1), (-1, 0)]:
            x_new, y_new = x + dx, y + dy
            if 0 <= x_new < x_max and 0 <= y_new < y_max and (dx, dy) != (0, 0):
                yield x_new, y_new

    # Dijkstraアルゴリズムを実行するメソッド
    def run(self, request_transitions=False, graph=None):
        while self.priority_queue:
            cur_cost, (cur_x, cur_y) = heapq.heappop(self.priority_queue)
            if (cur_x, cur_y) in self.certain:
                continue

            # 現在の頂点の近傍を探索
            for x, y in self.neighbors_func(cur_x, cur_y):
                if (x, y) not in self.certain:
                    if self.matrix[x][y] + self.costs[cur_x][cur_y] < self.costs[x][y]:
                        self.costs[x][y] = self.matrix[x][y] + self.costs[cur_x][cur_y]
                        heapq.heappush(self.priority_queue, (self.costs[x][y], (x, y)))
                        self.transitions[(x, y)] = (cur_x, cur_y)
                        self.num_path[x, y] = self.num_path[cur_x, cur_y]
                    elif self.matrix[x][y] + self.costs[cur_x][cur_y] == self.costs[x][y]:
                        self.num_path[x, y] += 1

            self.certain.add((cur_x, cur_y))

        # 最短経路を逆追跡
        cur_x, cur_y = self.x_max - 1, self.y_max - 1
        on_path = np.zeros_like(self.matrix)
        on_path[-1][-1] = 1
        edges = np.zeros(2 * 12 * 12 - 12 * 2)

        while (cur_x, cur_y) != (0, 0):
            if graph is not None:
                prev_x, prev_y = self.transitions[(cur_x, cur_y)]
                try:
                    edges[graph[tuple(sorted((prev_x * 12 + prev_y + 1, cur_x * 12 + cur_y + 1)))] - 1] = 1
                except:
                    import pdb; pdb.set_trace()
            cur_x, cur_y = self.transitions[(cur_x, cur_y)]
            on_path[cur_x, cur_y] = 1.0

        is_unique = self.num_path[-1, -1] == 1

        # DijkstraOutputを返す
        return DijkstraOutput(shortest_path=on_path, is_unique=is_unique, transitions=self.transitions if request_transitions else None, edges=edges)

# 実行例
if __name__ == "__main__":
    matrix = np.array([
        [1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]
    ])

    # 8方向の近傍を用いてDijkstraアルゴリズムを実行
    dijkstra = Dijkstra(matrix, neighbourhood_fn="8-grid")
    result = dijkstra.run()
    print("Shortest Path (8-grid):")
    print(result.shortest_path)
    print("Is Unique:", result.is_unique)

    # 4方向の近傍を用いてDijkstraアルゴリズムを実行
    dijkstra_4 = Dijkstra(matrix, neighbourhood_fn="4-grid")
    result_4 = dijkstra_4.run()
    print("Shortest Path (4-grid):")
    print(result_4.shortest_path)
    print("Is Unique:", result_4.is_unique)

Shortest Path (8-grid):
[[1 0 0]
 [0 1 0]
 [0 0 1]]
Is Unique: True
Shortest Path (4-grid):
[[1 1 1]
 [0 0 1]
 [0 0 1]]
Is Unique: True
